## Challenge
## In this module, we learned how to approach and solve regression problems using linear regression models. Throughout the module, you worked on a house price dataset from Kaggle. In this challenge, you will keep working on this dataset.

## The scenario
## The housing market is one of the most crucial parts of the economy for every country. Purchasing a home is one of the primary ways to build wealth and savings for people. In this respect, predicting prices in the housing market is a very central topic in economic and financial circles.

## The house price dataset from Kaggle includes several features of the houses along with their sale prices at the time they are sold. So far, in this module, you built and implemented some models using this dataset.

## In this challenge, you are required to improve your model with respect to its prediction performance.

## To complete this challenge, submit a Jupyter notebook containing your solutions to the following tasks.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import linear_model
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_error
from statsmodels.tools.eval_measures import mse, rmse

from sqlalchemy import create_engine

import warnings

warnings.filterwarnings('ignore')

## 1) Load the houseprices data from Thinkful's database.

In [2]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

house_df = pd.read_sql_query('select * from houseprices',con=engine)

# no need for an open connection, 
# as we're only doing a single query
engine.dispose()

In [3]:
house_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             1460 non-null   int64  
 1   mssubclass     1460 non-null   int64  
 2   mszoning       1460 non-null   object 
 3   lotfrontage    1201 non-null   float64
 4   lotarea        1460 non-null   int64  
 5   street         1460 non-null   object 
 6   alley          91 non-null     object 
 7   lotshape       1460 non-null   object 
 8   landcontour    1460 non-null   object 
 9   utilities      1460 non-null   object 
 10  lotconfig      1460 non-null   object 
 11  landslope      1460 non-null   object 
 12  neighborhood   1460 non-null   object 
 13  condition1     1460 non-null   object 
 14  condition2     1460 non-null   object 
 15  bldgtype       1460 non-null   object 
 16  housestyle     1460 non-null   object 
 17  overallqual    1460 non-null   int64  
 18  overallc

In [4]:
house_df.head(10)

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,...,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,12,2008,WD,Normal,250000
5,6,50,RL,85.0,14115,Pave,None,IR1,Lvl,AllPub,...,0,None,MnPrv,Shed,700,10,2009,WD,Normal,143000
6,7,20,RL,75.0,10084,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,8,2007,WD,Normal,307000
7,8,60,RL,NaN,10382,Pave,None,IR1,Lvl,AllPub,...,0,None,None,Shed,350,11,2009,WD,Normal,200000
8,9,50,RM,51.0,6120,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,4,2008,WD,Abnorml,129900
9,10,190,RL,50.0,7420,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,1,2008,WD,Normal,118000


In [5]:
house_df.describe()

,id,mssubclass,lotfrontage,lotarea,overallqual,overallcond,yearbuilt,yearremodadd,masvnrarea,bsmtfinsf1,...,wooddecksf,openporchsf,enclosedporch,threessnporch,screenporch,poolarea,miscval,mosold,yrsold,saleprice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


## 2) Do data cleaning, exploratory data analysis, and feature engineering. You can use your previous work in this module. But make sure that your work is satisfactory.

In [6]:
house_df = house_df[house_df['paveddrive'] != 'P']

In [7]:
house_df["has_central"] = pd.get_dummies(house_df['centralair'], drop_first=True)
house_df["has_paved"] = pd.get_dummies(house_df['paveddrive'], drop_first=True)

In [8]:
house_df.isna().sum().head(40)

id                 0
mssubclass         0
mszoning           0
lotfrontage      254
lotarea            0
street             0
alley           1346
lotshape           0
landcontour        0
utilities          0
lotconfig          0
landslope          0
neighborhood       0
condition1         0
condition2         0
bldgtype           0
housestyle         0
overallqual        0
overallcond        0
yearbuilt          0
yearremodadd       0
roofstyle          0
roofmatl           0
exterior1st        0
exterior2nd        0
masvnrtype         8
masvnrarea         8
exterqual          0
extercond          0
foundation         0
bsmtqual          36
bsmtcond          36
bsmtexposure      37
bsmtfintype1      36
bsmtfinsf1         0
bsmtfintype2      37
bsmtfinsf2         0
bsmtunfsf          0
totalbsmtsf        0
heating            0
dtype: int64

In [9]:
house_df['lotfrontage'] = house_df['lotfrontage'].fillna(house_df['lotfrontage'].mean())
house_df = house_df.drop(columns='alley')

In [10]:
house_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1430 entries, 0 to 1459
Data columns (total 82 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             1430 non-null   int64  
 1   mssubclass     1430 non-null   int64  
 2   mszoning       1430 non-null   object 
 3   lotfrontage    1430 non-null   float64
 4   lotarea        1430 non-null   int64  
 5   street         1430 non-null   object 
 6   lotshape       1430 non-null   object 
 7   landcontour    1430 non-null   object 
 8   utilities      1430 non-null   object 
 9   lotconfig      1430 non-null   object 
 10  landslope      1430 non-null   object 
 11  neighborhood   1430 non-null   object 
 12  condition1     1430 non-null   object 
 13  condition2     1430 non-null   object 
 14  bldgtype       1430 non-null   object 
 15  housestyle     1430 non-null   object 
 16  overallqual    1430 non-null   int64  
 17  overallcond    1430 non-null   int64  
 18  yearbuil

In [11]:
house_df['yearbuilt'].value_counts()

2006    67
2005    64
2004    54
2007    49
2003    45
        ..
1905     1
1906     1
1911     1
1942     1
1872     1
Name: yearbuilt, Length: 108, dtype: int64

In [12]:
house_df['miscfeature'].value_counts()

Shed    48
Othr     2
Gar2     2
TenC     1
Name: miscfeature, dtype: int64

In [13]:
house_df['neighborhood'].value_counts()

NAmes      224
CollgCr    150
OldTown    107
Edwards     93
Somerst     86
Gilbert     78
NridgHt     77
Sawyer      74
NWAmes      73
SawyerW     59
BrkSide     55
Crawfor     48
Mitchel     46
NoRidge     41
Timber      37
IDOTRR      34
ClearCr     28
StoneBr     25
SWISU       23
MeadowV     17
Blmngtn     17
BrDale      16
Veenker     11
NPkVill      9
Blueste      2
Name: neighborhood, dtype: int64

In [14]:
house_df['condition1'].value_counts()

Norm      1236
Feedr       79
Artery      45
RRAn        25
PosN        19
RRAe        11
PosA         8
RRNn         5
RRNe         2
Name: condition1, dtype: int64

In [15]:
house_df['condition2'].value_counts()

Norm      1416
Feedr        5
PosN         2
Artery       2
RRNn         2
PosA         1
RRAe         1
RRAn         1
Name: condition2, dtype: int64

In [16]:
house_df['bldgtype'].value_counts()

1Fam      1191
TwnhsE     114
Duplex      51
Twnhs       43
2fmCon      31
Name: bldgtype, dtype: int64

In [17]:
house_df['housestyle'].value_counts()

1Story    716
2Story    436
1.5Fin    145
SLvl       65
SFoyer     37
1.5Unf     13
2.5Unf     10
2.5Fin      8
Name: housestyle, dtype: int64

In [18]:
house_df['foundation'].value_counts()

PConc     641
CBlock    623
BrkTil    134
Slab       23
Stone       6
Wood        3
Name: foundation, dtype: int64

In [19]:
drop_cols = ['id', 'street', 'lotshape', 'landcontour', 'utilities', 'lotconfig', 'landslope', 'roofstyle', 'exterior1st',
            'exterior2nd', 'masvnrtype', 'garagetype', 'miscfeature', 'saletype', 'neighborhood', 'condition1', 'condition2',
            'bldgtype', 'housestyle', 'roofmatl', 'foundation']
house_df = house_df.drop(columns=drop_cols)

In [20]:
house_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1430 entries, 0 to 1459
Data columns (total 61 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   mssubclass     1430 non-null   int64  
 1   mszoning       1430 non-null   object 
 2   lotfrontage    1430 non-null   float64
 3   lotarea        1430 non-null   int64  
 4   overallqual    1430 non-null   int64  
 5   overallcond    1430 non-null   int64  
 6   yearbuilt      1430 non-null   int64  
 7   yearremodadd   1430 non-null   int64  
 8   masvnrarea     1422 non-null   float64
 9   exterqual      1430 non-null   object 
 10  extercond      1430 non-null   object 
 11  bsmtqual       1394 non-null   object 
 12  bsmtcond       1394 non-null   object 
 13  bsmtexposure   1393 non-null   object 
 14  bsmtfintype1   1394 non-null   object 
 15  bsmtfinsf1     1430 non-null   int64  
 16  bsmtfintype2   1393 non-null   object 
 17  bsmtfinsf2     1430 non-null   int64  
 18  bsmtunfs

In [21]:
bin_cols = []
house_df['has_central'].unique()

array([1, 0], dtype=uint8)

## 3) Now, split your data into train and test sets where 20% of the data resides in the test set.

In [22]:
X = house_df.drop['saleprice']
y = house_df['saleprice']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

TypeError: 'method' object is not subscriptable

## 4) Build several linear regression models including Lasso, Ridge, or ElasticNet and train them in the training set. Use k-fold cross-validation to select the best hyperparameters if your models include one!

## 5) Evaluate your best model on the test set.

## 6) So far, you have only used the features in the dataset. However, house prices can be affected by many factors like economic activity and the interest rates at the time they are sold. So, try to find some useful factors that are not included in the dataset. Integrate these factors into your model and assess the prediction performance of your model. Discuss the implications of adding these external variables into your model.